In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from statsmodels.tools.eval_measures import mse as sm_mse, rmse as sm_rmse, meanabs as sm_mae
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS, TFT, PatchTST, TCN, DLinear, RNN, LSTM, Autoformer, BiTCN, DeepAR, DeepNPTS, DilatedRNN, FEDformer, GRU, HINT, Informer, iTransformer, KAN, MLP, MLPMultivariate, NBEATSx, NLinear, RMoK, SOFTS, StemGNN, TiDE, TimeMixer, TimeLLM, TimesNet, TSMixer, TSMixerx, VanillaTransformer
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.losses.pytorch import MAE
from neuralforecast.utils import AirPassengersDF
import time
import pandas as pd
import numpy as np
import torch
torch.set_float32_matmul_precision('high')  # or 'medium' for more precision

import plotly.graph_objects as go
from sklearn.utils import check_random_state
import logging
logging.basicConfig(level=logging.INFO)
# Set seed for reproducibility
# Set up logging
logging.basicConfig(level=logging.INFO)

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random_state = check_random_state(seed)


In [2]:
req_naics = 336111
df = pd.read_csv(f"../data/processed_data_{req_naics}.csv")
df.head()

,year,naics,emp,pay,prode,prodh,prodw,vship,matcost,vadd,...,equip,plant,piship,pimat,piinv,pien,dtfp5,tfp5,dtfp4,tfp4
0,1958,336111,146.1,868.4,116.2,229.7,650.6,5007.9,3411.3,1563.8,...,3291.1,11718.8,0.314,0.238,0.180,0.147,NaN,0.553,NaN,0.550
1,1959,336111,160.6,1072.6,131.0,283.2,827.2,6422.2,4306.7,2143.4,...,3457.5,11415.6,0.322,0.243,0.184,0.145,0.052,0.583,0.053,0.580
2,1960,336111,176.1,1183.7,144.7,302.8,925.7,7239.0,4883.5,2336.2,...,3673.0,11375.1,0.318,0.241,0.190,0.150,0.030,0.601,0.030,0.598
3,1961,336111,152.4,1035.7,123.2,252.9,789.5,6214.2,4134.9,2053.6,...,3794.2,11179.9,0.317,0.242,0.190,0.146,-0.007,0.596,-0.009,0.592
4,1962,336111,168.2,1223.6,138.4,297.1,954.7,7855.3,5187.9,2671.2,...,3908.1,11141.2,0.315,0.246,0.193,0.147,0.077,0.644,0.078,0.641


In [3]:
# %% Convert 'Year' column to datetime and prepare multivariate dataset
df['ds'] = pd.to_datetime(df['year'], format='%Y')
df['unique_id'] = "336111"
df = df.rename(columns={"vship": "y"})  # Target variable
df.drop(columns=['year'], inplace=True)
feature_columns = [i for i in df.columns if i not in ['ds', 'y', 'unique_id']]
df = df.dropna()
df.head()

,naics,emp,pay,prode,prodh,prodw,y,matcost,vadd,invest,...,piship,pimat,piinv,pien,dtfp5,tfp5,dtfp4,tfp4,ds,unique_id
1,336111,160.6,1072.6,131.0,283.2,827.2,6422.2,4306.7,2143.4,93.0,...,0.322,0.243,0.184,0.145,0.052,0.583,0.053,0.580,1959-01-01,336111
2,336111,176.1,1183.7,144.7,302.8,925.7,7239.0,4883.5,2336.2,111.2,...,0.318,0.241,0.190,0.150,0.030,0.601,0.030,0.598,1960-01-01,336111
3,336111,152.4,1035.7,123.2,252.9,789.5,6214.2,4134.9,2053.6,91.8,...,0.317,0.242,0.190,0.146,-0.007,0.596,-0.009,0.592,1961-01-01,336111
4,336111,168.2,1223.6,138.4,297.1,954.7,7855.3,5187.9,2671.2,118.3,...,0.315,0.246,0.193,0.147,0.077,0.644,0.078,0.641,1962-01-01,336111
5,336111,160.6,1231.7,132.2,293.9,972.0,8254.4,5518.5,2748.9,144.9,...,0.313,0.245,0.195,0.147,0.014,0.653,0.014,0.649,1963-01-01,336111


In [4]:
# %% Train-test split
test_size = int(len(df) * 0.2)
train = df[:len(df) - test_size]
val = df[len(df) - test_size:]

print(f"Data split: Total: {len(df)}, Train: {len(train)}, Test: {len(val)}")
print("Training set date range:", train['ds'].min(), "to", train['ds'].max())
print("Testing set date range:", val['ds'].min(), "to", val['ds'].max())

Data split: Total: 58, Train: 47, Test: 11
Training set date range: 1959-01-01 00:00:00 to 2005-01-01 00:00:00
Testing set date range: 2006-01-01 00:00:00 to 2016-01-01 00:00:00


In [5]:
# %% Evaluation Function

def smape_loss(y_true, y_pred):
    y_true = y_true.values
    y_pred = y_pred.values
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

def evaluate_model(y_true, y_pred, y_train, model_name, start_time):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    smape = smape_loss(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    naive_forecast_errors = np.abs(y_train.diff()).dropna()
    mae_naive = sm_mae(naive_forecast_errors, np.zeros_like(naive_forecast_errors))
    mase = sm_mae(y_true, y_pred) / mae_naive if mae_naive != 0 else np.nan  
    runtime = time.time() - start_time

    return {
        "Model": model_name,
        "MSE": mse,
        "MAE": mae,
        "MAPE": mape,
        "SMAPE": smape,
        "MASE": mase,
        "R2 Score": r2,
        "Time (s)": runtime
    }


In [6]:
input_size =len(feature_columns) + 1

In [7]:
# %% Define Multivariate Models
models = [
    NBEATSx(h=12, input_size=24,  loss=MAE(), alias='NBEATSx_12'),
    TiDE(h=12, input_size=24, loss=MAE(), alias='TiDE_12'),
    TSMixer(h=12, input_size=24, n_series=len(feature_columns) + 1, loss=MAE(), alias='TSMixer_12'),
    #Informer(h=12, input_size=24,  loss=MAE(), alias='Informer_12'),
    #PatchTST(h=12, input_size=24, loss=MAE(), alias='PatchTST_12'),
]


Global seed set to 1
Global seed set to 1
Global seed set to 1


In [11]:
# %% Train & Predict
start_time = time.time()

for model in models:
    model.h = 12  # Adjust if necessary

nf = NeuralForecast(models=models, freq="Y")
nf.fit(train)
forecast_df = nf.predict()

# %% Evaluate Models
results = []
for model_name in forecast_df.columns:
    if model_name in ["ds", "unique_id"]:
        continue
    y_pred = forecast_df[model_name]
    y_pred = y_pred[:val["y"].shape[0]]
    results.append(evaluate_model(val["y"], y_pred, train["y"], f"{model_name} (NeuralForecast)", start_time))

# Convert results to DataFrame and sort by MAPE
df_results = pd.DataFrame(results).sort_values(by="MAPE")
print(df_results.head())



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 2.4 M 
-----------------------------------------------
2.4 M     Trainable params
900       Non-trainable params
2.4 M     Total params
9.789     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

ValueError: could not broadcast input array from shape (276,1) into shape (12,1)

In [9]:
print(nf.models[0].h)

12


In [ ]:
%tb

SIGTERMException: 

: 